In [3]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using StaticArrays 

#  Traversal(CTM::matrix,stack,obj,scene=[])

trasform the gerarchic in a unique coordinate system(world coordinate), the body of the obj is deepth-first searched and during the visit is used and updated the ctm matrix (by moltiplication the vertices(self multiplication))
during the visit we use the stack by pushing and poping the current ctm

In [4]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [5]:
structure1 = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "12693395839123810096", 2, "feature")

In [6]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [7]:
structure2 = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "13762169226783600592", 2, "feature")

In [8]:
Id = Matrix{Float64}(LinearAlgebra.I, 3, 3)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [20]:
function traversal(CTM::Matrix, stack, obj, scene=[])
	for i = 1:length(obj.body)
		if isa(obj.body[i],Matrix)
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal (generic function with 2 methods)

In [21]:
@btime traversal(Id,[],structure1,[])

  1.470 μs (16 allocations: 1.14 KiB)


1-element Vector{Any}:
 ([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [23]:
@btime traversal(Id,[],structure2,[])

  3.671 μs (25 allocations: 2.23 KiB)


2-element Vector{Any}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [24]:
@benchmark traversal(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.725 μs … 691.150 μs  ┊ GC (min … max): 0.00% … 98.16%
 Time  (median):     4.562 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.331 μs ±  11.104 μs  ┊ GC (mean ± σ):  3.50% ±  1.70%

      █▁                                                       
  ▂▃▃▅██▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  3.72 μs         Histogram: frequency by time          13 μs <

 Memory estimate: 2.23 KiB, allocs estimate: 25.

# Improvment & changes

In [34]:
function traversal(CTM::Matrix, stack, obj, scene)
	@inbounds for i in 1:length(obj.body)
		if (isa(obj.body[i],MMatrix) || isa(obj.body[i],Matrix))
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal (generic function with 2 methods)

In [35]:
@btime traversal(Id,[],structure2,[])

  3.725 μs (25 allocations: 2.23 KiB)


2-element Vector{Any}:
 ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])
 ([0.49999999999999994 -0.5 0.5 -0.49999999999999994; -0.5 -0.49999999999999994 0.49999999999999994 0.5], [[1, 2, 3, 4]])

In [36]:
@benchmark traversal(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.875 μs … 616.487 μs  ┊ GC (min … max): 0.00% … 98.14%
 Time  (median):     4.375 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.849 μs ±  10.118 μs  ┊ GC (mean ± σ):  3.56% ±  1.70%

    ▄▆█▅▁ ▂                                                    
  ▃▅███████▆▄▄▄▄▃▄▄▄▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▃
  3.88 μs         Histogram: frequency by time         8.2 μs <

 Memory estimate: 2.23 KiB, allocs estimate: 25.

# Paralelization

In [20]:
 function traversal(CTM::Matrix, stack, obj, scene)
	@inbounds @async for i in 1:length(obj.body)
		if (isa(obj.body[i],MMatrix) || isa(obj.body[i],Matrix))
			CTM = CTM*obj.body[i]
		elseif (isa(obj.body[i],Tuple) || isa(obj.body[i],Array)) && (length(obj.body[i])>=2)
			l = Lar.apply(CTM, obj.body[i])
			push!(scene,l)
		elseif isa(obj.body[i],Lar.Struct)
			push!(stack,CTM)
			Lar.traversal(CTM,stack,obj.body[i],scene)
			CTM = pop!(stack)
		end
	end
	return scene
end

traversal (generic function with 1 method)

In [21]:
@benchmark traversal(Id,[],structure2,[])

BenchmarkTools.Trial: 10000 samples with 196 evaluations.
 Range (min … max):  644.388 ns …  4.310 ms  ┊ GC (min … max):  0.00% … 99.96%
 Time  (median):       1.266 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.180 μs ± 50.065 μs  ┊ GC (mean ± σ):  35.81% ±  1.73%

      ▂▁▁▁▁▁ ▁▃▅▆██▇▇▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▂▂▃▂▃▂▂▂▁▁▁▁            ▃
  ▄█▇████████████████████████████████████████████████▇▇██▇▆▆▅▇ █
  644 ns        Histogram: log(frequency) by time      2.74 μs <

 Memory estimate: 704 bytes, allocs estimate: 8.